# Create args

In [ ]:
import argparse
import os 
class Args(argparse.Namespace):
    data="/scratch/guest187/Data/val_ssa/12_3d"
    preproc_set="val"
    data_used="SSA"
    results='/scratch/guest187/Results/train_ssa/ssa_split48/predictions_testing'
    exec_mode="predict"
    seed=42
    batch_size=4
    val_batch_size=2
    ckpt_path='/scratch/guest187/Results/train_ssa/ssa_split48/results/training48/f3/epoch=138-dice=90.01.ckpt'
    # model="brats22" #"unet"
    nfolds=1
    gpus=1 
    amp=True
    save_preds=True 
    brats=True
    brats22_model=True
    tta=True
args=Args()

In [1]:
import os

In [ ]:
import sys

In [1]:
import sys 
sys.path.append('/home/guest187/GitRepo_Brats23/nnUnet')
import subprocess
try:
    import os
    import numpy as np 
    import matplotlib.pyplot as plt
    import torch
    import torchio as tio
    import nibabel as nib
    from glob import glob
    from subprocess import call
    from scipy.ndimage import label
    from monai.inferers import sliding_window_inference
    from nnunet.brats22_model import UNet3D
    
    
except ModuleNotFoundError as e:
    package = str(e).split("'")[0]
    subprocess.run(['pip', 'install', package])

KeyboardInterrupt: 

In [2]:
import sys 
# sys.path.append('/Users/alexandrasmith/Desktop/Workspace/Projects/UNN_BraTS23/scripts')
sys.path.append('C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\UNN_BraTS23\\scripts')
import subprocess
try:
    import os
    import numpy as np 
    import matplotlib.pyplot as plt
    import torch
    import torchio as tio
    import nibabel as nib
    from glob import glob
    from subprocess import call
    from scipy.ndimage import label
    from monai.inferers import sliding_window_inference
    import modelZoo_monai as mZoo
    import data_loader as dl
    from monai_functions import (
        save_checkpoint,
        define_model,
        define_dataloaders,
        model_params,
        val_params,
        train)
    
except ModuleNotFoundError as e:
    package = str(e).split("'")[0]
    subprocess.run(['pip', 'install', package])

# Checkpoint

In [ ]:
pth = '/scratch/guest187/Data/train_gli_hack/results_hack_finetune_ssa/checkpoints/f0_finetune/epoch=39-dice=91.38.ckpt'
# pth = '/scratch/guest187/Data/train_all/results/test_fullRunThrough/best_metric_model_fullTest.pth'
checkpoint = torch.load(pth)
keys = checkpoint.keys()

for key in checkpoint.keys():
    print(key)

In [ ]:
state_dict = checkpoint["state_dict"]
print(type(state_dict))

state_dict2 = dict(state_dict)
print(type(state_dict2))


In [ ]:
import pickle
cnfgP = '/scratch/guest187/Data/train_all_nnUNET/train_all_data/results/11_3d/config.pkl'
config = pickle.load(open(cnfgP, "rb"))
print(config)

In [ ]:
patch_size, spacings = config["patch_size"], config["spacings"]
strides, kernels, sizes = [], [], patch_size[:]
while True:
    spacing_ratio = [spacing / min(spacings) for spacing in spacings]
    stride = [
        2 if ratio <= 2 and size >= 2 * 2 else 1 for (ratio, size) in zip(spacing_ratio, sizes)
    ]
    kernel = [3 if ratio <= 2 else 1 for ratio in spacing_ratio]
    if all(s == 1 for s in stride):
        break
    sizes = [i / j for i, j in zip(sizes, stride)]
    spacings = [i * j for i, j in zip(spacings, stride)]
    kernels.append(kernel)
    strides.append(stride)
    if len(strides) == 6:
        break
strides.insert(0, len(spacings) * [1])
kernels.append(len(spacings) * [3])

print(strides)
print(kernels)

# Setup

In [4]:
# define inference method
def inference(VAL_AMP, model, input):
    def _compute(input):
        '''
        roi_size – the spatial window size for inferences. 
        When its components have None or non-positives, the corresponding inputs dimension will be used. 
        if the components of the roi_size are non-positive values, the transform will use the corresponding components of img size.
        For example, roi_size=(32, -1) will be adapted to (32, 64) if the second spatial dimension size of img is 64
        '''
        return sliding_window_inference(
            inputs=input,
            roi_size=None,
            sw_batch_size=4, #sw_batch_size denotes the max number of windows per network inference iteration, not the batch size of inputs.
            predictor=model,
            overlap=0.2,
            mode='constant'
        )
    if VAL_AMP:
        with torch.cuda.amp.autocast():
            return _compute(input)
    else:
        return _compute(input)


def infer():
    with torch.no_grad():
        for step, batch in enumerate(val_loader):
            input = batch[0]
            # case = str(batch[1]).split('/')[-1].replace("-stk.npy',)", "")
            case = str(batch[1]).split('\\')[-1].replace("-stk.npy',)", "")
            print(f"Working on subject: {case}; shape size is: {input.shape}")
            #Run inference
            output = inference(VAL_AMP, model, input)
            print(f"Output before post_transforms is: {output.shape}")
            val_output = post_transforms(output[0])
            print(f"Output after post_transforms is: {val_output.shape}")

            #Attempt to convert and save to numpy for post processing
            val_output2 = val_output.cpu().numpy()
            val_out_pth = os.path.join(args.results, f"{case}_preds.npy")
            try:
                print(f"trying to save to numpy file, with shape: {val_output2.shape}")
                np.save(val_out_pth, val_output2)
            except Exception as e:
                print(f"An error occurred while saving the file: {e}")

# Save to nifti
def save_nifti(final_np, img, outfile):
    '''
    Save numpy to NIfTI format
    input numpy must have 3 dimensions only
    '''
    
    img = nib.load(img) ## load original image from path
    nib.save(
        nib.Nifti1Image(final_np, img.affine, header=img.header),
        outfile
    )

def save_final_preds(pred, img, outfile, fname):
    #convert tensor to tio subject for padding and affine alignment
    predT = torch.from_numpy(pred).unsqueeze(0)
    subject = tio.Subject(
            image=tio.ScalarImage(tensor=predT),
            name=fname
            )
    tranformed_subject = tio.CropOrPad((240, 240, 155))
    tranformed_subject = tranformed_subject(subject)
    pred_final = tranformed_subject["image"].data.numpy().squeeze(0)
    print(f"Shape chanes: initial = {pred.shape}, tensor = {predT.shape}, and final npy = {pred_final.shape}")

    save_nifti(pred_final, img, outfile)


#------------ OPTINET -----##
def to_lbl(pred):
    print("in to_lbl: ", pred.shape)
    print(pred.shape)
    print(pred[0].shape, pred[1].shape, pred[2].shape)
# labels_dict = {"0": "background", "1": "edema", "2": "non-enhancing tumor", "3": "enhancing tumour"}

# # The segementation volume contains values [0, 1, 2, 3]. As per BraTS summarizing paper for 2023, annotations comprise the 
# - 1 for NCR (necrotic tumor core)
# - 2 for ED (peritumoral edematous/invaded tissue)
# - 3 for ET (GD-enhancing tumor)
# - 0 for everything else

    enh = pred[2]
    print(enh.shape)
    pad = pred == 0.5
    pred[pad==True] = 0

    c1, c2, c3 = pred[0] > 0.4, pred[1] > 0.35, pred[2] > 0.375
    print("CShapes: ", c1.shape, c2.shape, c3.shape)
    # print(set(pred.flatten()))
    print("before assignment:" ,pred.shape)

    # print(set(pred.flatten()))
    pred = (c1 > 0).astype(np.uint8)
    # print(set(pred.flatten()))
        #  assigns 0 to elements in pred where c1 is False and 1 where c1 is True, effectively converting c1 to 0s and 1s in pred
            ## if the WT probability for a given voxel is less than 0.45 then its class is set to 0 (background) or its assigned to NCR

    pred[(c2 == False) * (c1 == True)] = 2 #AA changed to 1
    # print(set(pred.flatten()))
        # This step assigns 2 to regions where c2 is False and c1 is True, which might indicate a region that extends beyond the threshold of c1 and is within the threshold of c2.
            # otherwise if the probability for TC is less than 0.4 the voxel class is 2 (ED) (wherever NCR was assigned in C1 as 1, if TC is false at that point, then class is oedema)

    pred[(c3 == True) * (c1 == True)] = 3
    # print(set(pred.flatten()))
        # assigns 3 to regions where c3 is True and c1 is True, which might indicate a region that is within the threshold of both c1 and c3.
        # finally if probability for ET is less than 0.4 voxel has class 1 (NCR), or otherwise 4 (ET).

    print("after assignment: ", pred.shape)


    components, n = label(pred == 3)
    for et_idx in range(1, n + 1):
        _, counts = np.unique(pred[components == et_idx], return_counts=True)
        if 1 < counts[0] and counts[0] < 4 and np.mean(enh[components == et_idx]) < 0.9:
            pred[components == et_idx] = 1
    print("after for loop: ", pred.shape)
    
    et = pred == 3
    if 0 < et.sum() and et.sum() < 5 and np.mean(enh[et]) < 0.9:
        pred[et] = 1
    print("before _lbl return: ", pred.shape)

    pred = np.transpose(pred, (2, 1, 0)).astype(np.uint8)
    print("after transpose: ", pred.shape)
    return pred

def prepare_preditions(e):
    fname = e.split("\\")[-1].split(".")[0]
    # fname = e.split("\\")[-1].split(".")[0][:-6]
    print(fname)
    # CALL BELOW LINE FOR POST PROCESSING with ensembling
    # preds = [np.load(f) for f in e]
    # print(preds)
    # p = to_lbl(np.mean(preds, 0))

    # ELSE JUST SAVE NP MEANS ALONG AXIS 0
    pred = np.load(e)
    # print(pred.shape)

    p = to_lbl(pred)
    print(p.shape)

    return p

In [ ]:
validation_dir=args.data
validation_files = [os.path.join(validation_dir, file) for file in os.listdir(validation_dir)]
print('\n'.join(validation_files))

model, n_channels = define_model(args.ckpt_path)

# Load validation data to dataloader
data_transforms = dl.define_transforms(n_channels)
dataloader = dl.load_data(args, data_transforms)

val_loader = dataloader['val']

# Validation parameters
VAL_AMP, dice_metric, dice_metric_batch, post_transforms = val_params()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()
infer()


In [ ]:
max_idx = 15
for step, batch in enumerate(val_loader):
    if step < max_idx:
        input = batch[0]
        print(input.shape)
        print(str(batch[1]).split('/')[-1].replace("-stk_x.npy',)", ""))

In [ ]:
max_idx = 15
with torch.no_grad():
    for step, batch in enumerate(val_loader):
        if step < max_idx:
            input = batch[0].to(device)
            case = str(batch[1]).split('/')[-1].replace("-stk_x.npy',)", "")
            print(input.shape)
            output = inference(VAL_AMP, model, input)
            print(output.shape)
            val_output = post_transforms(output[0])
            print(val_output.shape)
            
            val_output2 = val_output.cpu().numpy()
            print(type(val_output2))

            val_out_pth = os.path.join(args.results, f"{case}_preds.npy")
            print(type(val_out_pth))
            try:
                np.save(val_out_pth, val_output2)
            except Exception as e:
                print(f"An error occurred while saving the file: {e}")
            # visualize the 3 channels model output corresponding to this image
            plt.figure("model output", (18, 6))
            for i in range(3):
                plt.subplot(1, 3, i + 1)
                plt.title(f"{case}: output channel {i}")
                plt.imshow(val_output2[i, :, :, 30])
            plt.show()

In [ ]:
print(device)

In [8]:
results = 'C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\train_gli\\ftSSA_f0\\'
try:
    os.makedirs(os.path.join(results, "final_preds"))
except:
    print("directory exists")

#--------------MONAI
# examples = sorted(glob('C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\npy\\*'))
# orig_data = 'C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\Data\\SSA_Val'
orig_data = 'C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\CC\\Backup_2407\\val_SSA\\monai'
print(os.path.exists(orig_data))

#------------- OPTINET     
preds = sorted(glob('C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\train_gli\\ftSSA_f0\\predictions*'))
# examples = list(zip(*[sorted(glob(f"{p}\\*.npy")) for p in preds]))
examples = [sorted(glob(f"{p}\\*.npy")) for p in preds]
print(examples)

print("Preparing final predictions")
for e in examples:
    print(e)
    for pnpy in e:
        pred = prepare_preditions(pnpy)
        fname = pnpy.split("\\")[-1].split(".")[0]
        # # fname = e.split("\\")[-1].split(".")[0][:-6]
        # # print(pred.shape)
        outfile = os.path.join(results,"final_preds", f"{fname}.nii.gz")
        img = os.path.join(orig_data, fname, f"{fname}-t1n.nii.gz")
        img_load = nib.load(img)
        nib.save(
            nib.Nifti1Image(pred, img_load.affine, header=img_load.header),
        outfile
        )
    # save_final_preds(pred, img, outfile, fname)
    
print("Completed!")

True
[['C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\train_gli\\ftSSA_f0\\predictions_epoch=98-dice=95_04_task=01_fold=0_tta\\BraTS-SSA-00126-000.npy', 'C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\train_gli\\ftSSA_f0\\predictions_epoch=98-dice=95_04_task=01_fold=0_tta\\BraTS-SSA-00129-000.npy', 'C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\train_gli\\ftSSA_f0\\predictions_epoch=98-dice=95_04_task=01_fold=0_tta\\BraTS-SSA-00132-000.npy', 'C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\train_gli\\ftSSA_f0\\predictions_epoch=98-dice=95_04_task=01_fold=0_tta\\BraTS-SSA-00139-000.npy', 'C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\train_gli\\ftSSA_f0\\predictions_epoch=98-dice=95_04_task=01_fold=0_tta\\BraTS-SSA-00143-000.npy', 'C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\train_gli\\ftSSA_f0\\predictions_epoch=98-dice=95_04_task=01_fold=0_tta\\BraTS-SSA-00148-000.npy', 'C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\

In [ ]:
sample = nib.load(os.path.join (args.results, "BraTS-SSA-00132-000.nii.gz"))
print(sample.shape)

print(set(np.array(sample.dataobj).flatten()))

In [ ]:
lbl_chk = np.load('C:\\Users\\amoda\\Documents\\SPARK\\BraTS2023\\results\\preds_allFolds\\predictions_epoch=128-dice=93_74_task=01_fold=3_tta\\BraTS-SSA-00143-000.npy')

print("Brain background: ", lbl_chk[:,70,89,76])
print("pad background: ",lbl_chk[:,70,227,216] )
print("Yellow: ",lbl_chk[:,70,113,178])
print("Green: ",lbl_chk[:,70,134,150])
print("Red: ",lbl_chk[:,70,142,121])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from glob import glob

n, z = 5, 75
data = sorted(glob("/scratch/guest187/Data/val_SSA/results/monai_test/*.nii.gz"))
for i in range(n):
    fname = data[i].split("/")[-1].split(".")[0]
    print(fname)
    img = nib.load(f"/scratch/guest187/Data/val_SSA/finetune_ssa/images/{fname}-stk.nii.gz").get_fdata().astype(np.float32)
    pred = nib.load(data[i]).get_fdata().astype(np.uint8)[:, :, z]
    imgs = [img[:, :, z, i] for i in [0, 3]] + [pred]
    
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 12))
    for i in range(3):
        if i < 2:
            ax[i].imshow(imgs[i], cmap='gray')
        else:
            ax[i].imshow(imgs[i]);
        ax[i].axis('off')  
    plt.tight_layout()            
    plt.show()